In [11]:
!pip install dash plotly pandas

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import dash_table
from dash import Dash, dcc, html, Input, Output, State

# Load the gyroscope data from a CSV file
df = pd.read_csv('gyroscope_data.csv')

# Initialize the Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Gyroscope Data Visualization"),
    
    # Dropdown for graph type selection
    dcc.Dropdown(
        id='graph-type',
        options=[
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Line Chart', 'value': 'line'},
            {'label': 'Distribution Plot', 'value': 'histogram'}
        ],
        value='scatter'
    ),
    
    # Dropdown for selecting x, y, z variables
    dcc.Dropdown(
        id='data-variables',
        options=[
            {'label': 'X-Axis', 'value': 'x'},
            {'label': 'Y-Axis', 'value': 'y'},
            {'label': 'Z-Axis', 'value': 'z'},
            {'label': 'All', 'value': 'all'}
        ],
        value='all',
        multi=True
    ),
    
    # Input box for the number of samples
    html.Label("Number of Samples:"),
    dcc.Input(id='num-samples', type='number', value=100, min=1),
    
    # Navigation buttons for previous and next samples
    html.Button('Previous', id='prev-button', n_clicks=0),
    html.Button('Next', id='next-button', n_clicks=0),
    
    # Graph to display the data
    dcc.Graph(id='gyro-graph'),
    
    # Table to summarize the data
    html.Div(id='summary-table')
])

@app.callback(
    Output('gyro-graph', 'figure'),
    Output('summary-table', 'children'),
    Input('graph-type', 'value'),
    Input('data-variables', 'value'),
    Input('num-samples', 'value'),
    Input('prev-button', 'n_clicks'),
    Input('next-button', 'n_clicks')
)
def update_graph(graph_type, variables, num_samples, prev_clicks, next_clicks):
    # Determine the start index based on navigation
    start_index = max(0, (next_clicks - prev_clicks) * num_samples)
    
    # Filter the data to display
    filtered_df = df.iloc[start_index:start_index + num_samples]
    
    # Define a function to create the figure
    def create_figure():
        fig = go.Figure()
        
        if graph_type == 'scatter':
            if 'all' in variables:
                for var in ['accel_x', 'accel_y', 'accel_z']:
                    fig.add_trace(go.Scatter(x=filtered_df.index, y=filtered_df[var], mode='markers', name=var))
            else:
                fig.add_trace(go.Scatter(x=filtered_df.index, y=filtered_df[variables[0]], mode='markers'))
        
        elif graph_type == 'line':
            if 'all' in variables:
                for var in ['accel_x', 'accel_y', 'accel_z']:
                    fig.add_trace(go.Scatter(x=filtered_df.index, y=filtered_df[var], mode='lines', name=var))
            else:
                fig.add_trace(go.Scatter(x=filtered_df.index, y=filtered_df[variables[0]], mode='lines'))
        
        elif graph_type == 'histogram':
            if 'all' in variables:
                for var in ['accel_x', 'accel_y', 'accel_z']:
                    fig.add_trace(go.Histogram(x=filtered_df[var], name=var, opacity=0.75))
                fig.update_layout(barmode='overlay')
            else:
                fig.add_trace(go.Histogram(x=filtered_df[variables[0]]))

        return fig

    # Create the figure
    fig = create_figure()

    # Create summary table
    selected_columns = ['accel_x', 'accel_y', 'accel_z'] if 'all' in variables else variables
    summary = filtered_df[selected_columns].describe().reset_index()
    summary_table = dash_table.DataTable(
        data=summary.to_dict('records'),
        columns=[{"name": i, "id": i} for i in summary.columns]
    )
    
    return fig, summary_table

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # Change 8051 to any other available port
